<a href="https://colab.research.google.com/github/Aman2632/basic_programming/blob/master/leaf_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss

In [18]:
train_raw= pd.read_csv('train.csv')
test_raw = pd.read_csv('test.csv')

print("Size of  the training  data:",train_raw.shape[0],"*",train_raw.shape[1])
print("Size of  the test  data:",test_raw.shape[0],"*",test_raw.shape[1])

Size of  the training  data: 990 * 194
Size of  the test  data: 594 * 193


In [19]:
le= LabelEncoder().fit(train_raw.species) #encoding  the species 
labels= le.transform(train_raw.species)
classes= list(le.classes_)
test_ids= test_raw.id
train=  train_raw.drop(['id','species'], axis= 1)
test= test_raw.drop(['id'], axis =1)


In [20]:
#from sklearn.model_selection import train_test_split

ss_split = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
ss_split.get_n_splits(train, labels)

for train_index, test_index in ss_split.split(train, labels):   
    X_train, X_test = train.values[train_index], train.values[test_index]
    y_train, y_test = labels[train_index], labels[test_index]


In [22]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

param_grid = {'C': [1000, 10000],'tol': [0.000001, 0.00001]}
log_reg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
grid_search = GridSearchCV(log_reg, param_grid, scoring= 'neg_log_loss', refit='True', n_jobs=1, cv=None)
grid_search.fit(X_train_scaled, y_train)

print("Best parameters: {}".format(grid_search.best_params_))
print('Best neg_loss_score: {}'.format(grid_search.best_score_))
print('Best estimator: {}'.format(grid_search.best_estimator_))


Best parameters: {'C': 10000, 'tol': 1e-05}
Best neg_loss_score: -0.048449995350367016
Best estimator: LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=1e-05, verbose=0,
                   warm_start=False)


In [25]:
scaler = StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)

train_predictions = grid_search.predict(X_test_scaled)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
train_predictions_prob = grid_search.predict_proba(X_test_scaled)
logloss = log_loss(y_test, train_predictions_prob)
print("Log Loss: {:.6}".format(logloss))

Accuracy: 98.4848%
Log Loss: 0.0236682


In [27]:
# GENERATING CSV OUTPUT FILE ON THE test.csv 


scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

# Using the optimal parameters
param_grid = {'C': [1000],
              'tol': [0.000001]}
log_reg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
grid_search = GridSearchCV(log_reg, param_grid, scoring='neg_log_loss', refit='True', n_jobs=1, cv=ss_split)
grid_search.fit(X_train_scaled, y_train)

scaler = StandardScaler().fit(test)
test_scaled = scaler.transform(test)

test_predictions = grid_search.predict_proba(test_scaled)

# Format DataFrame
submission = pd.DataFrame(test_predictions, columns=classes)
submission.insert(0, 'id', test_ids)
submission.reset_index()

# Export Submission
submission.to_csv('output.csv', index = False)

# Double check the output
submission.head()

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,Acer_Saccharinum,Alnus_Cordata,Alnus_Maximowiczii,Alnus_Rubra,Alnus_Sieboldiana,Alnus_Viridis,Arundinaria_Simonii,Betula_Austrosinensis,Betula_Pendula,Callicarpa_Bodinieri,Castanea_Sativa,Celtis_Koraiensis,Cercis_Siliquastrum,Cornus_Chinensis,Cornus_Controversa,Cornus_Macrophylla,Cotinus_Coggygria,Crataegus_Monogyna,Cytisus_Battandieri,Eucalyptus_Glaucescens,Eucalyptus_Neglecta,Eucalyptus_Urnigera,Fagus_Sylvatica,Ginkgo_Biloba,Ilex_Aquifolium,Ilex_Cornuta,Liquidambar_Styraciflua,Liriodendron_Tulipifera,Lithocarpus_Cleistocarpus,Lithocarpus_Edulis,...,Quercus_Coccinea,Quercus_Crassifolia,Quercus_Crassipes,Quercus_Dolicholepis,Quercus_Ellipsoidalis,Quercus_Greggii,Quercus_Hartwissiana,Quercus_Ilex,Quercus_Imbricaria,Quercus_Infectoria_sub,Quercus_Kewensis,Quercus_Nigra,Quercus_Palustris,Quercus_Phellos,Quercus_Phillyraeoides,Quercus_Pontica,Quercus_Pubescens,Quercus_Pyrenaica,Quercus_Rhysophylla,Quercus_Rubra,Quercus_Semecarpifolia,Quercus_Shumardii,Quercus_Suber,Quercus_Texana,Quercus_Trojana,Quercus_Variabilis,Quercus_Vulcanica,Quercus_x_Hispanica,Quercus_x_Turneri,Rhododendron_x_Russellianum,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,9.008434e-11,1.110463e-09,2.632073e-13,3.716450e-08,2.352777e-09,4.064803e-09,1.768521e-13,2.364594e-12,7.386875e-12,3.676975e-08,7.082984e-08,1.145264e-13,4.401491e-13,3.473237e-10,6.500442e-09,7.234471e-10,2.587518e-06,1.598682e-13,1.033758e-08,1.517595e-11,6.732377e-08,1.251436e-09,8.497245e-08,6.052109e-13,5.473392e-12,2.648303e-11,3.560764e-06,7.648078e-08,7.793212e-11,8.371970e-11,3.361997e-18,3.471217e-06,1.155666e-06,1.362437e-06,4.886900e-12,5.866732e-08,3.324809e-10,3.928827e-20,2.679459e-09,...,9.744338e-11,1.072914e-09,4.635259e-10,7.671141e-10,2.469741e-10,1.604761e-05,1.097404e-09,1.227928e-07,3.012206e-17,3.546980e-12,1.040004e-10,2.725857e-13,5.151118e-12,3.497276e-07,5.337441e-15,1.026236e-05,3.092328e-06,2.656604e-15,5.048058e-09,1.314882e-05,1.828030e-10,1.270991e-08,1.541359e-15,1.340223e-17,3.356001e-05,2.508164e-05,8.666719e-14,3.622023e-15,1.999738e-16,5.108456e-06,5.026842e-15,8.301392e-10,2.904141e-10,1.291162e-11,2.987240e-09,7.928672e-13,1.093352e-13,3.406073e-15,3.417424e-07,7.521494e-12
1,7,7.569856e-10,6.803398e-09,7.776958e-08,5.441388e-07,1.546202e-09,9.544596e-09,7.877243e-06,1.120744e-09,1.172221e-09,9.564221e-09,3.287150e-12,6.188733e-11,3.321578e-11,3.397008e-10,9.572006e-10,5.723148e-10,4.091255e-10,2.001442e-07,4.845402e-10,9.255324e-07,4.929411e-09,2.344501e-11,9.017100e-10,9.167035e-11,6.606083e-10,2.909519e-10,1.573706e-08,1.096748e-07,2.968103e-10,2.293852e-12,7.074956e-09,3.541063e-09,8.101210e-09,1.320970e-10,6.828909e-08,2.554701e-11,6.898415e-06,4.270850e-10,4.699562e-08,...,3.178571e-07,6.385420e-08,1.278245e-08,4.069405e-10,1.406554e-07,3.554263e-13,5.023459e-10,3.999385e-10,2.565569e-09,3.404363e-08,6.651070e-06,5.086648e-13,7.474035e-11,1.937686e-10,5.969287e-09,7.216791e-15,4.613882e-11,1.596871e-06,1.680182e-09,2.513140e-09,1.253934e-08,9.922973e-10,5.827232e-09,2.141518e-07,5.451317e-07,5.720239e-07,1.377167e-07,1.272074e-07,2.963321e-07,1.231878e-13,1.600169e-08,2.606813e-08,4.108339e-11,2.318302e-12,1.294124e-14,1.906134e-06,2.574062e-09,3.042251e-06,2.758030e-10,1.193339e-08
2,9,5.304588e-08,9.973271e-01,1.104389e-07,3.858415e-08,2.148125e-03,3.744998e-06,2.818599e-08,8.377288e-06,5.785608e-05,2.687391e-05,4.376429e-08,4.034032e-05,4.035905e-08,1.450595e-07,4.266312e-06,2.731486e-06,2.302545e-04,3.418138e-10,9.727925e-08,5.910047e-06,8.279505e-08,1.566042e-07,4.686104e-11,2.194924e-10,4.265368e-11,1.112154e-05,8.998275e-06,1.523493e-10,6.898161e-09,1.393938e-09,1.575823e-10,1.512741e-08,4.819382e-06,2.001269e-06,1.109232e-11,5.432313e-06,8.455850e-10,6.104561e-12,4.110559e-10,...,1.423864